In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

In [38]:
twitter_auth_token = "dfe219d71b6bf946e51a906ea073abfda45e5d6b"

def setup_driver() -> WebDriver:
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    driver.get("https://x.com")
    driver.add_cookie({
        'name': 'auth_token',
        'value': twitter_auth_token,
        'domain': '.x.com'
    })
    
    return driver

driver = setup_driver()

username = input("Masukkan username Twitter: ")
url = f"https://x.com/{username}/with_replies"
driver.get(url)

try:
    WebDriverWait(driver, 40).until(
        EC.presence_of_element_located((By.XPATH, "//article"))
    )
except Exception as e:
    print("Error: Elemen tweet tidak ditemukan", e)

tweets_collected = set()
while len(tweets_collected) < 50:
    time.sleep(3)  
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
    
    tweets = soup.find_all("article")
    
    for tweet in tweets:
        user_element = tweet.find("a", href=f"/{username}") 
        if user_element:
            tweet_text_element = tweet.find("div", class_=lambda x: x and "r-bnwqim" in x)
            if tweet_text_element:
                tweet_text = tweet_text_element.get_text(strip=True)
                tweets_collected.add(tweet_text)
                if len(tweets_collected) >= 50:
                    break
    
    driver.execute_script("window.scrollBy(0, 3000);") 

df_tweets = pd.DataFrame(list(tweets_collected), columns=["Tweet"])

# Tutup browser
driver.quit()

In [39]:
df_tweets

,Tweet
0,
1,Wweheheh
2,"Ndak lah kalau advertising, langsung saja tunj..."
3,Proses setelah 6jam didalam frezerr. \nMixer s...
4,Kirim kan photo tumbuhan apa yg berbuah depan ...
5,Sami sami
6,InsyaAllah ada waktunya
7,Diluar baliho narsis Ormas atau tokoh. Yg sang...
8,Bangetan.
9,Nanti malam kita buktikan ya.


In [41]:
df.Tweet.duplicated().sum()

np.int64(0)

In [22]:
df.to_excel('cobaTwitterProfile.xlsx', index=False)